In [ ]:
### local DB setup and load
import os
import yaml
import pandas as pd
from dotenv import load_dotenv

from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

print(load_dotenv("../chatbot_poc_playground/.env"))

True


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_and_chunk_pdfs(pdf_dir):

    all_chunks = []

    for filename in os.listdir(pdf_dir):
        if filename.lower().endwith("pdf"):
            file_path = os.path.join(pdf_dir, filename)
            print(f"Loading file: {filename}")
            pages = load_pdf_file(file_path)
            chunks = split_pdf_pages_into_chunks(pdf_pages=pages, 
                                        chunk_size=400, 
                                        chunk_overlap=500)
            all_chunks.extend(chunks)            
            return    


def load_pdf_file(file_path):
    loader = PyPDFLoader(file_path)
    # Load pages
    pages = loader.load()
    return pages

def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()  # Read entire file as a single string
    return content.strip()

def split_text_into_chunks(text, chunk_size=3000, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    chunks = splitter.split_text(text)
    return chunks

def split_pdf_pages_into_chunks(pdf_pages, chunk_size=3000, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    chunks = splitter.split_documents(pdf_pages)    
    return chunks

# file_path = "../sample_data/ntrs-10k-2023.pdf"
# pdf_pages = load_pdf_file(file_path)

file_path = "../sample_data/ntrs-20231231.htm.txt"
text_data = load_text_file(file_path)

txt_chunks = split_text_into_chunks(text_data)
print(f"Split text into {len(txt_chunks)} chunk")
print(txt_chunks[0])




Writing test.py
